In [2]:
import os
from train import CVAE_FULL
import keras
from keras import backend as K
import tensorflow as tf
import numpy as np
import gait
from tqdm import tqdm
import cv2
from train_birnn import model_BiRNN, ImageNoise
from sklearn.ensemble import RandomForestClassifier 